# 8.2.2 Building a RAG Application with Aurora PostgreSQL and pgvector

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#Step-1-Install-Required-Libraries">Step 1: Install Required Libraries</a></li>
<li><a href="#Step-2-Connect-to-Aurora-PostgreSQL">Step 2: Connect to Aurora PostgreSQL</a></li>
<li><a href="#Step-3-Enable-pgvector-Extension-and-Create-Tables">Step 3: Enable pgvector Extension and Create Tables</a></li>
<li><a href="#Step-4-Create-HNSW-Index-for-Efficient-Vector-Search">Step 4: Create HNSW Index for Efficient Vector Search</a></li>
<li><a href="#Step-5-Create-Bedrock-Knowledge-Base-with-Web-Crawler">Step 5: Create Bedrock Knowledge Base with Web Crawler</a></li>
<li><a href="#Step-6-Query-the-Bedrock-Knowledge-Base">Step 6: Query the Bedrock Knowledge Base</a></li>
<li><a href="#Step-7-Create-Additional-Tables-for-Advanced-Queries">Step 7: Create Additional Tables for Advanced Queries</a></li>
<li><a href="#Step-8-Implement-Vector-Search-with-pgvector">Step 8: Implement Vector Search with pgvector</a></li>
<li><a href="#Step-9-Advanced-Query-Optimization-Techniques">Step 9: Advanced Query Optimization Techniques</a></li>
<li><a href="#Step-10-Complete-RAG-Pipeline-with-LLM-Response-Generation">Step 10: Complete RAG Pipeline with LLM Response Generation</a></li>
<li><a href="#Step-11-Compare-Both-RAG-Approaches">Step 11: Compare Both RAG Approaches</a></li>
</ul>
</div>


In this notebook, we'll build a complete [RAG (Retrieval Augmented Generation)](https://aws.amazon.com/what-is/retrieval-augmented-generation/) application using [Amazon Aurora PostgreSQL](https://aws.amazon.com/rds/aurora/postgresql-features/) with [pgvector](https://github.com/pgvector/pgvector) for vector storage and [Amazon Bedrock](https://aws.amazon.com/bedrock/) for embedding generation and LLM capabilities.

RAG combines the power of retrieval-based systems with generative AI to produce more accurate, contextually relevant, and factual responses. By using Aurora PostgreSQL with pgvector, we can efficiently store and query vector embeddings, while Bedrock provides the foundation models for generating embeddings and responses.

## What You'll Learn
- How to set up pgvector in Aurora PostgreSQL for vector similarity search
- How to create and optimize vector indexes for performance
- How to integrate with Amazon Bedrock for embeddings and LLM responses
- How to implement advanced query optimization techniques for vector search

## Prerequisites
- AWS account with [Amazon Bedrock](https://aws.amazon.com/bedrock/) and [Aurora](https://aws.amazon.com/rds/aurora/) access
- Aurora PostgreSQL cluster (you can use one created in [Section 2.1](../../2_Your_First_Database_on_AWS/2.1_Crearting_Your_First_Aurora_Cluster/README.MD) or create a new one)
- Jupyter Notebook: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- Basic understanding of SQL and Python

## Step 1: Install Required Libraries

We'll start by installing the necessary Python libraries for our RAG application:
- [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html): AWS SDK for Python to interact with AWS services
- [psycopg2](https://www.psycopg.org/docs/): PostgreSQL adapter for Python
- [langchain](https://python.langchain.com/docs/get_started/introduction): Framework for developing applications powered by language models
- [requests](https://requests.readthedocs.io/en/latest/): HTTP library for making API calls
- [beautifulsoup4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/): Library for web scraping

In [ ]:
# Install required libraries
!pip install -q boto3 psycopg2-binary langchain requests beautifulsoup4

## Step 2: Connect to Aurora PostgreSQL

Next, we'll establish a connection to our Aurora PostgreSQL database using credentials stored in [AWS Secrets Manager](https://aws.amazon.com/secrets-manager/). This approach follows security best practices by avoiding hardcoded credentials in our code.

The process involves:
1. Retrieving the secret containing database credentials
2. Extracting connection parameters from the secret
3. Establishing a connection to the Aurora PostgreSQL database

In [ ]:
import psycopg2
import boto3
import json

# Initialize the AWS Secrets Manager client
secrets_client = boto3.client('secretsmanager')

# Specify the name of your Aurora PostgreSQL secret
secret_name = "aurora-postgresql-secret"  # Replace with your actual secret name

try:
    # Retrieve the secret
    response = secrets_client.get_secret_value(SecretId=secret_name)
    secret = json.loads(response['SecretString'])
    
    # Extract database connection parameters from the secret
    db_host = secret['host']
    db_port = secret.get('port', 5432)
    db_name = secret.get('dbname', 'postgres')
    db_user = secret['username']
    db_password = secret['password']
    
    print(f"Successfully retrieved secret for {secret_name}")
    
    # Connect to the database using the retrieved credentials
    conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_name,
        user=db_user,
        password=db_password
    )
    cursor = conn.cursor()
    
    print("Connected to the database successfully!")
    
except Exception as e:
    print(f"Error retrieving secret or connecting to database: {e}")

## Step 3: Enable pgvector Extension and Create Tables

Now we'll set up [pgvector](https://github.com/pgvector/pgvector), an open-source PostgreSQL extension that enables vector similarity search. This extension allows us to store embeddings as a native data type and perform efficient similarity searches.

We'll perform the following tasks:
1. Enable the pgvector extension in our database
2. Verify the installed pgvector version
3. Create a table for storing documents and their vector embeddings

The `documents` table will include a `VECTOR(1536)` column to store embeddings from the Titan Embeddings model, which produces 1536-dimensional vectors.

In [ ]:
# Enable pgvector extension
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")

# Check pgvector version
cursor.execute("SELECT extversion FROM pg_extension WHERE extname = 'vector';")
pgvector_version = cursor.fetchone()[0]
print(f"pgvector version: {pgvector_version}")

# Create a table for documents and their embeddings
cursor.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    title TEXT NOT NULL,
    content TEXT NOT NULL,
    url TEXT,
    category TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    embedding VECTOR(1536)
);
""")

conn.commit()
print("Database schema created successfully!")

## Step 4: Create HNSW Index for Efficient Vector Search

[HNSW (Hierarchical Navigable Small World)](https://arxiv.org/abs/1603.09320) indexes provide faster query performance compared to traditional [IVF (Inverted File)](https://en.wikipedia.org/wiki/Inverted_index) indexes, especially for high-dimensional vectors. HNSW is an approximate nearest neighbor search algorithm that creates a multi-layered graph structure for efficient navigation.

Key HNSW parameters we're configuring:
- `m = 16`: Controls the maximum number of connections per node in the graph
- `ef_construction = 64`: Controls the size of the dynamic candidate list during index construction

These settings balance index build time, search performance, and memory usage. Higher values generally provide better search quality at the cost of increased memory usage and build time.

In [ ]:
# Create an HNSW index for efficient vector search
cursor.execute("""
CREATE INDEX IF NOT EXISTS documents_embedding_hnsw_idx 
ON documents 
USING hnsw (embedding vector_cosine_ops)
WITH (m = 16, ef_construction = 64);
""")

conn.commit()
print("HNSW index created successfully!")

## Step 5: Create Bedrock Knowledge Base with Web Crawler

In this step, we'll create an [Amazon Bedrock Knowledge Base](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) that uses our Aurora PostgreSQL database as its vector store. A Knowledge Base in Bedrock provides a managed solution for implementing RAG applications.

We'll perform the following tasks:
1. Create an IAM role with necessary permissions for Bedrock
2. Configure the Knowledge Base to use Aurora PostgreSQL as the vector store
3. Set up a web crawler data source to ingest AWS documentation
4. Monitor the ingestion process

Using the [web crawler](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-create-datasource.html#knowledge-base-create-datasource-web) capability, we can automatically extract, process, and index content from the Aurora PostgreSQL documentation website.

In [ ]:
import boto3
import time
import uuid

# Initialize Bedrock clients
bedrock = boto3.client('bedrock')
agents = boto3.client('bedrock-agent')

# Define knowledge base parameters
kb_name = "aurora-postgresql-kb"
kb_description = "Knowledge base for Aurora PostgreSQL documentation"
embedding_model_id = "amazon.titan-embed-text-v2:0"

# Create a unique service role name
role_name = f"bedrock-kb-role-{uuid.uuid4().hex[:8]}"

# Create IAM role for Bedrock Knowledge Base
iam = boto3.client('iam')

# Create the IAM role with trust policy
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "bedrock.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    role_response = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Role for Bedrock Knowledge Base",
        Tags=[
            {
                'Key': 'CreationSource',
                'Value': 'aws-database-cookbook-v2025.8'
            }
        ]
    )
    
    # Attach necessary policies
    iam.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
    )
    
    # Wait for role to propagate
    print("Waiting for IAM role to propagate...")
    time.sleep(10)
    
    role_arn = role_response['Role']['Arn']
    print(f"Created role: {role_arn}")
    
except Exception as e:
    print(f"Error creating role: {e}")
    # If role already exists, get its ARN
    role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    print(f"Using existing role: {role_arn}")

In [ ]:
# Create the knowledge base with Aurora PostgreSQL as vector store
try:
    # Get Aurora PostgreSQL connection details
    rds_data_config = {
        "databaseConnectionConfiguration": {
            "rdsConfiguration": {
                "host": db_host,
                "port": db_port,
                "databaseName": db_name,
                "credentials": {
                    "secretArn": f"arn:aws:secretsmanager:{boto3.session.Session().region_name}:{boto3.client('sts').get_caller_identity()['Account']}:secret:{secret_name}"
                }
            }
        }
    }
    
    # Create knowledge base with Aurora PostgreSQL as vector store
    kb_response = agents.create_knowledge_base(
        name=kb_name,
        description=kb_description,
        roleArn=role_arn,
        knowledgeBaseConfiguration={
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": f"arn:aws:bedrock:{boto3.session.Session().region_name}::foundation-model/{embedding_model_id}"
            }
        },
        storageConfiguration={
            "type": "RDS",
            "rdsStorageConfiguration": rds_data_config
        },
        tags={
            'CreationSource': 'aws-database-cookbook-v2025.8'
        }
    )
    
    kb_id = kb_response['knowledgeBase']['knowledgeBaseId']
    print(f"Created knowledge base with ID: {kb_id}")
    
    # Wait for knowledge base to be active
    print("Waiting for knowledge base to become active...")
    while True:
        kb_status = agents.get_knowledge_base(knowledgeBaseId=kb_id)['knowledgeBase']['status']
        print(f"Knowledge base status: {kb_status}")
        if kb_status == "ACTIVE":
            break
        time.sleep(10)
    
except Exception as e:
    print(f"Error creating knowledge base: {e}")

In [ ]:
# Add web crawler data source
try:
    # URL of Aurora PostgreSQL documentation
    url = "https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.AuroraPostgreSQL.html"
    
    # Create data source with web crawler
    data_source_response = agents.create_data_source(
        knowledgeBaseId=kb_id,
        name="aurora-postgresql-docs",
        description="Aurora PostgreSQL documentation from AWS docs",
        dataSourceConfiguration={
            "type": "WEB",
            "webConfiguration": {
                "url": url,
                "crawlerConfiguration": {
                    "crawlMode": "SUBPAGES",
                    "crawlDepth": 2,  # Crawl up to 2 levels deep
                    "maxUrls": 20,    # Limit to 20 URLs for demo purposes
                }
            }
        },
        vectorIngestionConfiguration={
            "chunkingConfiguration": {
                "chunkingStrategy": "FIXED_SIZE",
                "fixedSizeChunkingConfiguration": {
                    "maxTokens": 300,
                    "overlapPercentage": 20
                }
            }
        }
    )
    
    data_source_id = data_source_response['dataSource']['dataSourceId']
    print(f"Created data source with ID: {data_source_id}")
    
    # Start data source ingestion
    ingestion_job = agents.start_ingestion_job(
        knowledgeBaseId=kb_id,
        dataSourceId=data_source_id
    )
    
    ingestion_job_id = ingestion_job['ingestionJob']['ingestionJobId']
    print(f"Started ingestion job with ID: {ingestion_job_id}")
    
except Exception as e:
    print(f"Error creating data source or starting ingestion: {e}")

In [ ]:
# Monitor ingestion job status
def check_ingestion_status(kb_id, data_source_id, ingestion_job_id):
    try:
        response = agents.get_ingestion_job(
            knowledgeBaseId=kb_id,
            dataSourceId=data_source_id,
            ingestionJobId=ingestion_job_id
        )
        
        status = response['ingestionJob']['status']
        metrics = response['ingestionJob'].get('metrics', {})
        
        print(f"Ingestion status: {status}")
        if metrics:
            print(f"Documents processed: {metrics.get('documentsProcessed', 'N/A')}")
            print(f"Documents failed: {metrics.get('documentsFailed', 'N/A')}")
            print(f"Vectors ingested: {metrics.get('vectorsIngested', 'N/A')}")
        
        return status
    except Exception as e:
        print(f"Error checking ingestion status: {e}")
        return None

# Check status every 30 seconds for up to 10 minutes
for _ in range(20):
    status = check_ingestion_status(kb_id, data_source_id, ingestion_job_id)
    if status in ["COMPLETE", "FAILED"]:
        break
    print("Waiting 30 seconds...")
    time.sleep(30)

print("Ingestion process completed or timed out.")

## Step 7: Create Additional Tables for Advanced Queries

To demonstrate the power of combining vector search with traditional relational database capabilities, we'll create additional tables for metadata and tagging. This [relational model](https://en.wikipedia.org/wiki/Relational_model) allows us to perform more sophisticated queries that combine semantic similarity with structured filtering.

We'll create:
1. A `document_metadata` table to store additional information about documents (author, publication date, version, etc.)
2. A `document_tags` table to implement a [many-to-many relationship](https://en.wikipedia.org/wiki/Many-to-many_(data_model)) between documents and tags
3. Appropriate indexes to optimize query performance

This approach demonstrates how to leverage both the vector capabilities of pgvector and the relational capabilities of PostgreSQL in a single application.

In [ ]:
import boto3
import json

# Initialize Bedrock client for embeddings
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # Change to your region
)

def get_embedding(text):
    """Generate embedding using Amazon Titan Embeddings v2 model"""
    response = bedrock_runtime.invoke_model(
        modelId='amazon.titan-embed-text-v2:0',  # Using v2 model as specified
        contentType='application/json',
        accept='application/json',
        body=json.dumps({
            "inputText": text
        })
    )
    response_body = json.loads(response['body'].read())
    return response_body['embedding']

# Create metadata table for additional document information
cursor.execute("""
CREATE TABLE IF NOT EXISTS document_metadata (
    document_id INTEGER PRIMARY KEY REFERENCES documents(id),
    author TEXT,
    publication_date DATE,
    version TEXT,
    importance_score FLOAT,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

# Create table for document tags
cursor.execute("""
CREATE TABLE IF NOT EXISTS document_tags (
    id SERIAL PRIMARY KEY,
    document_id INTEGER REFERENCES documents(id),
    tag TEXT NOT NULL,
    UNIQUE(document_id, tag)
);
""")

# Create index on document_tags for faster lookups
cursor.execute("""
CREATE INDEX IF NOT EXISTS document_tags_tag_idx ON document_tags(tag);
""")

conn.commit()
print("Additional tables created successfully!")

# Add sample metadata and tags for existing documents
cursor.execute("SELECT id, title FROM documents LIMIT 10")
existing_docs = cursor.fetchall()

for doc_id, title in existing_docs:
    # Add metadata
    cursor.execute(
        """INSERT INTO document_metadata 
           (document_id, author, publication_date, version, importance_score) 
           VALUES (%s, %s, %s, %s, %s)
           ON CONFLICT (document_id) DO NOTHING""",
        (doc_id, "AWS Documentation Team", "2023-01-01", "1.0", 0.8)
    )
    
    # Add tags based on title
    tags = ["Aurora", "PostgreSQL"]
    if "performance" in title.lower():
        tags.append("Performance")
    if "security" in title.lower():
        tags.append("Security")
    if "feature" in title.lower() or "features" in title.lower():
        tags.append("Features")
    
    for tag in tags:
        cursor.execute(
            """INSERT INTO document_tags (document_id, tag) 
               VALUES (%s, %s) 
               ON CONFLICT (document_id, tag) DO NOTHING""",
            (doc_id, tag)
        )

conn.commit()
print("Added metadata and tags to existing documents!")

## Step 6: Query the Bedrock Knowledge Base

Now let's create functions to query our Bedrock Knowledge Base and generate responses using the LLM. This demonstrates the [RAG pattern](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) where we:

1. Retrieve relevant information from the Knowledge Base based on a user query
2. Augment the LLM prompt with this retrieved context
3. Generate a response that's grounded in the retrieved information

We'll use the [Bedrock Agent Runtime API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock_Agent_Runtime.html) to query the Knowledge Base and the [Bedrock Runtime API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_Operations_Amazon_Bedrock_Runtime.html) to generate responses using Claude or another foundation model.

In [ ]:
# Function to query the knowledge base
def query_knowledge_base(kb_id, query_text, max_results=3):
    try:
        # Initialize the Bedrock Agent Runtime client
        bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
        
        # Query the knowledge base
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={
                'text': query_text
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results
                }
            }
        )
        
        return response['retrievalResults']
    except Exception as e:
        print(f"Error querying knowledge base: {e}")
        return []

# Function to generate RAG response using Bedrock model
def generate_kb_response(kb_id, query_text, model_id="anthropic.claude-v2"):
    # Step 1: Retrieve relevant chunks from knowledge base
    results = query_knowledge_base(kb_id, query_text)
    
    if not results:
        return "Unable to retrieve information from the knowledge base."
    
    # Step 2: Build context from retrieved chunks
    context = "\n\n".join([result['content']['text'] for result in results])
    
    # Step 3: Generate response using Bedrock model
    bedrock_runtime = boto3.client('bedrock-runtime')
    
    prompt = f"""
    Human: Answer the following question based on the provided context.
    If you cannot answer the question based on the context, say "I don't have enough information to answer this question."
    
    Context:
    {context}
    
    Question: {query_text}
    
    Assistant:
    """
    
    try:
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                "prompt": prompt,
                "max_tokens_to_sample": 500,
                "temperature": 0.7,
                "top_p": 0.9,
            })
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['completion']
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response from the model."

# Test the RAG pipeline with our knowledge base
query = "What are the key features of Aurora PostgreSQL?"
print(f"Query: {query}\n")

# Replace with your actual knowledge base ID
response = generate_kb_response(kb_id, query)
print(f"Response: {response}")

## Step 8: Implement Vector Search with pgvector

Now we'll implement a custom vector search function using pgvector's capabilities. This demonstrates how to perform [vector similarity search](https://www.pinecone.io/learn/vector-similarity/) directly in PostgreSQL without relying on external vector databases.

The process involves:
1. Converting a text query into a vector embedding using Bedrock's embedding model
2. Using pgvector's `<=>` operator to calculate [cosine distance](https://en.wikipedia.org/wiki/Cosine_similarity) between the query embedding and document embeddings
3. Retrieving the most similar documents based on this distance metric

The `1 - (embedding <=> %s)` expression converts cosine distance to cosine similarity, which ranges from 0 (completely dissimilar) to 1 (identical).

In [ ]:
def vector_search(query_text, top_k=3):
    """Perform vector similarity search"""
    # Generate embedding for the query
    query_embedding = get_embedding(query_text)
    
    # Perform vector similarity search
    cursor.execute("""
    SELECT title, content, url, 1 - (embedding <=> %s) as similarity
    FROM documents
    ORDER BY embedding <=> %s
    LIMIT %s
    """, (query_embedding, query_embedding, top_k))
    
    results = cursor.fetchall()
    return results

# Test the search
query = "What are the key features of Aurora PostgreSQL?"
search_results = vector_search(query)

print(f"Query: {query}\n")
print("Top results:")
for title, content, url, similarity in search_results:
    print(f"Title: {title}")
    print(f"Similarity: {similarity:.4f}")
    print(f"URL: {url}")
    print("-" * 50)

## Step 9: Advanced Query Optimization Techniques

Now let's explore some advanced [query optimization techniques](https://aws.amazon.com/blogs/database/optimize-query-performance-with-pgvector-in-amazon-aurora-postgresql/) for pgvector. These techniques can significantly improve search performance, especially for large datasets or complex queries.

We'll cover:
- Iterative index scans for filtered queries
- Creating proper indexes for filtering conditions
- Partial indexing and partitioning for specialized workloads

These optimizations are particularly important for production RAG applications where response time is critical.

### 9.1 Iterative Index Scans (pgvector 0.8.0+)

[Iterative index scans](https://github.com/pgvector/pgvector/blob/master/README.md#query-options) automatically scan more of the index until enough results are found, which is particularly useful for filtered queries. This feature was introduced in pgvector 0.8.0 and provides a significant performance improvement for queries that combine vector similarity with traditional SQL filters.

The `ef_search` parameter controls how many candidates are considered during the search. Higher values increase accuracy but decrease performance. We'll demonstrate this technique by implementing an advanced search function that combines vector similarity with metadata filtering and joins across multiple tables.

In [ ]:
# Example of iterative index scan with filtering and joins
def advanced_vector_search(query_text, tag=None, min_importance=0.5, top_k=3):
    """Perform filtered vector similarity search with iterative scanning and joins"""
    query_embedding = get_embedding(query_text)
    
    # Set ef_search parameter for iterative scanning
    cursor.execute("SET hnsw.ef_search = 100;")
    
    # Build the query based on filters
    query = """
    SELECT d.title, d.content, d.url, 1 - (d.embedding <=> %s) as similarity,
           m.author, m.importance_score, array_agg(t.tag) as tags
    FROM documents d
    JOIN document_metadata m ON d.id = m.document_id
    LEFT JOIN document_tags t ON d.id = t.document_id
    WHERE m.importance_score >= %s
    """
    
    params = [query_embedding, min_importance]
    
    # Add tag filter if provided
    if tag:
        query += "AND EXISTS (SELECT 1 FROM document_tags WHERE document_id = d.id AND tag = %s)"
        params.append(tag)
    
    # Complete the query
    query += """
    GROUP BY d.id, d.title, d.content, d.url, d.embedding, m.author, m.importance_score
    ORDER BY d.embedding <=> %s
    LIMIT %s
    """
    
    params.extend([query_embedding, top_k])
    
    # Execute the query
    cursor.execute(query, params)
    results = cursor.fetchall()
    return results

# Test advanced search with joins
advanced_results = advanced_vector_search(
    "What are the key features of Aurora PostgreSQL?", 
    tag="Features", 
    min_importance=0.7
)

print("Advanced search results with joins:")
for title, content, url, similarity, author, importance, tags in advanced_results:
    print(f"Title: {title}")
    print(f"Author: {author}")
    print(f"Importance: {importance:.2f}")
    print(f"Tags: {', '.join(tags)}")
    print(f"Similarity: {similarity:.4f}")
    print("-" * 50)

### 9.2 Creating Proper Indexes for Filtering

When combining vector search with filtering, it's important to create appropriate [indexes](https://www.postgresql.org/docs/current/indexes.html) on the filter columns. PostgreSQL offers several index types, each optimized for different query patterns:

- [B-tree indexes](https://www.postgresql.org/docs/current/btree-intro.html): The default index type, good for equality and range queries
- [GIN indexes](https://www.postgresql.org/docs/current/gin-intro.html): Generalized Inverted Indexes, excellent for full-text search
- [BRIN indexes](https://www.postgresql.org/docs/current/brin-intro.html): Block Range INdexes, efficient for large tables with naturally ordered data

We'll create these different index types to optimize various filtering scenarios in our RAG application.

In [ ]:
# Create B-tree index on category column
cursor.execute("""
CREATE INDEX IF NOT EXISTS documents_category_idx 
ON documents (category);
""")

# Create GIN index on title for full-text search
cursor.execute("""
CREATE INDEX IF NOT EXISTS documents_title_idx 
ON documents USING gin(to_tsvector('english', title));
""")

# Create BRIN index on created_at for range queries
cursor.execute("""
CREATE INDEX IF NOT EXISTS documents_created_at_idx 
ON documents USING brin(created_at);
""")

conn.commit()
print("Additional indexes created successfully!")

### 9.3 Partial Indexing and Partitioning

For better performance with specific filtering patterns, we can use [partial indexes](https://www.postgresql.org/docs/current/indexes-partial.html) or [table partitioning](https://www.postgresql.org/docs/current/ddl-partitioning.html). These advanced PostgreSQL features can significantly improve query performance for large datasets:

- **Partial Indexes**: Create indexes that only include rows matching a specific condition, reducing index size and maintenance overhead
- **Table Partitioning**: Divide large tables into smaller, more manageable pieces based on specific criteria (e.g., category)

We'll demonstrate both techniques using our document collection, creating a partial index for Aurora PostgreSQL-specific documents and implementing list partitioning by category.

In [ ]:
# Create a partial index for a specific category
cursor.execute("""
CREATE INDEX IF NOT EXISTS documents_aurora_idx 
ON documents (id) 
WHERE category = 'Aurora PostgreSQL';
""")

# Example of creating a partitioned table (for demonstration only)
cursor.execute("""
CREATE TABLE IF NOT EXISTS documents_partitioned (
    id SERIAL,
    title TEXT NOT NULL,
    content TEXT NOT NULL,
    url TEXT,
    category TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    embedding VECTOR(1536),
    PRIMARY KEY (category, id)
) PARTITION BY LIST (category);
""")

# Create a partition for Aurora PostgreSQL documents
try:
    cursor.execute("""
    CREATE TABLE documents_aurora PARTITION OF documents_partitioned
    FOR VALUES IN ('Aurora PostgreSQL');
    """)
    print("Partition created successfully!")
except psycopg2.errors.DuplicateTable:
    print("Partition already exists.")
    conn.rollback()
else:
    conn.commit()

## Step 10: Complete RAG Pipeline with LLM Response Generation

Now we'll implement a complete [RAG pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) using our custom pgvector implementation. This pipeline combines vector search with LLM response generation to create a system that can answer questions based on our document collection.

The pipeline consists of three main steps:
1. **Retrieval**: Find relevant documents using vector similarity search
2. **Augmentation**: Enhance the prompt with retrieved context
3. **Generation**: Generate a response using a foundation model from [Amazon Bedrock](https://aws.amazon.com/bedrock/)

This approach helps ground the LLM's responses in factual information from our document collection, reducing hallucinations and improving accuracy.

In [ ]:
def generate_pgvector_response(query):
    """Complete RAG pipeline with pgvector search and LLM response generation"""
    # Step 1: Retrieve relevant documents
    search_results = vector_search(query, top_k=3)
    
    if not search_results:
        return "No relevant documents found in the database."
    
    # Step 2: Build context from retrieved documents
    context = "\n\n".join([content for _, content, _, _ in search_results])
    
    # Step 3: Generate response using Amazon Bedrock
    prompt = f"""
    Human: Answer the following question based on the provided context.
    If you cannot answer the question based on the context, say "I don't have enough information to answer this question."
    
    Context:
    {context}
    
    Question: {query}
    
    Assistant:
    """
    
    try:
        response = bedrock_runtime.invoke_model(
            modelId='anthropic.claude-v2',  # Or your preferred model
            contentType='application/json',
            accept='application/json',
            body=json.dumps({
                "prompt": prompt,
                "max_tokens_to_sample": 500,
                "temperature": 0.7,
                "top_p": 0.9,
            })
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['completion']
    except Exception as e:
        print(f"Error generating response: {e}")
        return "Error generating response from the model."

# Test the complete pgvector RAG pipeline
query = "What are the benefits of using Aurora PostgreSQL?"
response = generate_pgvector_response(query)
print(f"Query: {query}\n")
print(f"Response: {response}")

## Step 11: Compare Both RAG Approaches

Let's compare the results from both our RAG implementations - the custom pgvector approach and the [Bedrock Knowledge Base](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) approach. This comparison will help us understand the trade-offs between:

1. **Custom Implementation**: More control and flexibility, but requires more development effort
2. **Managed Service**: Easier to set up and maintain, but potentially less customizable

We'll use the same query for both approaches and compare the quality, relevance, and accuracy of the responses. This will help you decide which approach is better suited for your specific use case.

In [ ]:
# Compare both RAG approaches with the same query
test_query = "What are the key features and benefits of Aurora PostgreSQL?"

print(f"Query: {test_query}\n")
print("=== pgvector RAG Response ===")
pgvector_response = generate_pgvector_response(test_query)
print(pgvector_response)

print("\n" + "=" * 50 + "\n")

print("=== Bedrock Knowledge Base RAG Response ===")
kb_response = generate_kb_response(kb_id, test_query)
print(kb_response)

## Summary

In this notebook, we've built a complete [RAG application](https://aws.amazon.com/what-is/retrieval-augmented-generation/) using [Amazon Aurora PostgreSQL](https://aws.amazon.com/rds/aurora/postgresql-features/) with [pgvector](https://github.com/pgvector/pgvector) and [Amazon Bedrock](https://aws.amazon.com/bedrock/). We've covered:

1. Setting up pgvector in Aurora PostgreSQL for vector similarity search
2. Creating and optimizing [vector indexes](https://github.com/pgvector/pgvector#indexing) (HNSW) for performance
3. Creating a [Bedrock Knowledge Base](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html) with web crawler for document ingestion
4. Implementing [vector search](https://www.pinecone.io/learn/vector-similarity/) for semantic document retrieval
5. Advanced query optimization techniques:
   - [Iterative index scans](https://github.com/pgvector/pgvector/blob/master/README.md#query-options) for filtered queries
   - [Proper indexing](https://www.postgresql.org/docs/current/indexes.html) for filtering conditions
   - [Partial indexing](https://www.postgresql.org/docs/current/indexes-partial.html) and [partitioning](https://www.postgresql.org/docs/current/ddl-partitioning.html) for specialized workloads
6. Building complete RAG pipelines with both approaches:
   - Custom pgvector implementation for maximum flexibility
   - Managed Bedrock Knowledge Base for ease of use

These techniques provide a solid foundation for building production-ready RAG applications with optimal performance and cost efficiency. By leveraging the power of Aurora PostgreSQL's relational capabilities combined with pgvector's vector search functionality, you can create sophisticated AI applications that deliver accurate, contextually relevant responses.

For more information, check out these resources:
- [Amazon Aurora PostgreSQL Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.AuroraPostgreSQL.html)
- [pgvector GitHub Repository](https://github.com/pgvector/pgvector)
- [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [AWS RAG Reference Implementation](https://github.com/aws-samples/rag-with-amazon-bedrock)